In [3]:
import numpy as np
import pandas as pd
import uuid

## Описание проекта и предметной области
Я бывший администратор баз данных Oracle с большим опытом и знаниями многих аспектов работы БД.

В арсенал инструментов администраторов БД Oracle (DBA) есть портал Oracle Enterprise Manager (OEM), который позволяет управлять работой многих систем, построенных преимущественно на основе технологий Oracle. Среди прочего функционала OEM собирает и хранит для последующего анализа информацию о конфигурациях компонентов. 
Информации действительно много и она затрагивает почти все особенности ежесекундной работы БД Oracle. В самом OEM есть инструструменты для анализа и визуализации этих данных, но сам сервис немного перегружен функционалом и может ощутимо медленно работать.

Мне всегда было интересно, как эти данные можно использовать для анализа и прогнозирования работы данных. Но обычно DBA не заходит дальше, чем построить некий график на период времени в прошлом, что мало информативное и обычно выглядит как мельтешение кривых. В последних версиях Oracle сам пытается внедрить интеллектуальный анализ статистики, но я это не смотрел, потому что ушел от администрирования БД.

Сейчас у меня есть доступ до БД мониторинга тестовых сред. Я сделал исследование, как это храниться внутри и как можно удобно выгрузить для импорта в pandas.

Запрос, которым посмотрел все метрики, которые доступны, всего их более 200:

`select distinct metric_name, metric_column, metric_label from sysman.mgmt$metric_daily`

Работа по определению что за что отвечает и что показывает достойна отдельного исследования, потому что официального описания внутренней структуры БД OEM нет.

In [2]:
import os
print(os.listdir("../input"))

['sample_redosize1.csv', 'metrics.csv', 'database_original.csv', 'sample_dbtime2.csv', 'sample_dbsize1.csv', 'sample_dbsize2.csv', 'sample_dbtime1.csv', 'database_data.csv']


In [3]:
metrict = pd.read_csv("../input/metrics.csv", decimal=',', index_col=False)

In [4]:
metrict.sample(5)

,metric_name,metric_column,metric_label
14,instance_throughput,physreads_pt,Throughput
46,instance_throughput,enqtimeouts_ps,Throughput
36,instance_throughput,rollbacks_pt,Throughput
131,instance_throughput,logreads_ps,Throughput
175,instance_throughput,physreads_ps,Throughput


Выбрал несколько метрик, для примера — это объем БД в Гб(SIZE_GB), объем изменений в Мб(REDOSIZE_MB), средняя нагрузка (DBTIME) по дням. Данные есть за несколько лет, с 2017 года примерно.

Запрос для формирования финальной выгрузки:

`SELECT database,
       target_guid,
       rollup_timestamp,
       ROUND (SUM (DECODE (metric_column, 'redosize_pt', maximum)) / 1024 / 1024, 3) redosize_gb,
       ROUND (SUM (DECODE (metric_column, 'dbtime_pt', maximum)), 3) dbtime,
       ROUND (SUM (DECODE (metric_column, 'USED_GB', maximum)), 3) used_gb
  FROM (SELECT target_name database,
               target_guid,
               rollup_timestamp,
               key_value tb,
               metric_column,
               ROUND (MAX (maximum), 0) maximum
          FROM sysman.mgmt$metric_daily
         WHERE     metric_name IN ('instance_throughput', 'DATABASE_SIZE')
               AND metric_column IN ('redosize_pt', 'dbtime_pt', 'USED_GB')
        GROUP BY target_name,
                 target_guid,
                 key_value,
                 rollup_timestamp,
                 metric_column)
GROUP BY database, target_guid, rollup_timestamp
ORDER BY database, target_guid, rollup_timestamp
/`

## Обезличивание исходных данных и подготовка нескольких примеров

Исходные данные имеют значения для имени БД в формате SID_имя_сервера, что может быть расценено как конфиденциальная инфомация, поэтому заменяю на рандомные uuid'ы.

In [19]:
database_original = pd.read_csv("./input/tablespace_original.csv", decimal=',', index_col=False

In [20]:
lowercase = lambda x: str(x).lower()
database_original.rename(lowercase, axis="columns", inplace=True)
database_original['rollup_timestamp'] = pd.to_datetime(database_original['rollup_timestamp'], format = '%d.%m.%Y')
database_original = database_original.set_index('rollup_timestamp')

In [5]:
uuids = [str(uuid.uuid4()) for x in range(len(database_original['database'].drop_duplicates()))]

In [6]:
deperson_map = dict(zip(database_original['database'].drop_duplicates(), uuids))

In [8]:
database_data = database_original.copy()
database_data['database'] = database_data['database'].map(deperson_map)

In [9]:
tablespace_data.to_csv('../input/database_data.csv')

### Итоговый данные

In [4]:
database_data = pd.read_csv("../input/database_data.csv", decimal=',', index_col=False)
database_data = database_data.set_index('rollup_timestamp')

In [13]:
database_data

,database,redosize_mb,dbtime,size_gd
rollup_timestamp,,,,
2019-04-11,e859c352-1087-4602-a410-77845a1ad2f6,606.0,83.0,396.0
2019-04-12,e859c352-1087-4602-a410-77845a1ad2f6,2328.0,649.0,824.0
2019-04-13,e859c352-1087-4602-a410-77845a1ad2f6,1221.0,33.0,232.0
2019-04-14,e859c352-1087-4602-a410-77845a1ad2f6,246.0,661.0,107.0
2019-04-15,e859c352-1087-4602-a410-77845a1ad2f6,1610.0,665.0,504.0
...,...,...,...,...
2020-03-18,fe4dd1a0-9bf2-40d6-8950-d52c9af1c0f7,175.0,673.0,41.0
2020-03-19,fe4dd1a0-9bf2-40d6-8950-d52c9af1c0f7,698.0,276.0,42.0
2020-03-20,fe4dd1a0-9bf2-40d6-8950-d52c9af1c0f7,576.0,941.0,41.0


In [11]:
database_data.shape

(293572, 5)